In [1]:
from puku_core.graphs.knowledge_trees.nodes.markdown import MarkdownNode
from puku_core.graphs.knowledge_trees.edges.markdown import MarkdownEdge
from puku_core.documents.markdown import parse

from puku.graphs.knowledge_trees.writers.chat_model import ChatModelAmendmentSplitter

In [2]:
root = MarkdownNode(
    data=parse(
        """
Duonychus (meaning "two claws") is an extinct genus of therizinosaurid theropod dinosaurs that lived during the Late Cretaceous (Cenomanian to early Coniacian age) of what is now Mongolia. It is known from a partial skeleton, including several vertebrae, most of the forelimbs, and part of the pelvic girdle, found in outcrops of the Bayanshiree Formation. The remains were discovered in 2012 by the Mongolian Academy of Sciences and briefly mentioned in later conference abstracts. The genus contains a single species, Duonychus tsogtbaatari, which was formally described in 2025.

The hand of Duonychus only has two fingers, similar to tyrannosaurids, compared to the three commonly found in most theropods. This anatomical feature—in addition to its long, strongly curved claws—may have allowed Duonychus to efficiently grasp plant material to consume. The geological formation from which Duonychus is known has also yielded the therizinosaurs Enigmosaurus, Erlikosaurus, and Segnosaurus, in addition to many other dinosaurs.                         """
    ),
    title="Duonychus",
    description="Main information about Duonychus",
)

In [3]:
descr = MarkdownNode(
    data=parse(
        """
The Duonychus holotype belongs to an immature individual with an estimated body mass of approximately 260 kilograms (570 lb), similar in size to Erlikosaurus, which is around 278 kilograms (613 lb). In comparison, the other coeval Enigmosaurus and Segnosaurus were larger, at 567 kilograms (1,250 lb) and 1,469 kilograms (3,239 lb), respectively.[1] Although no estimate of body length was given in the academic description of the taxon in 2025, the accompanying press release noted that the animal would have reached approximately 3 metres (9.8 ft) long.[4]
    """
    ),
    title="description",
    description="How does it look",
)

In [4]:
discovery = MarkdownNode(
    data=parse(
        """
In 2012, the Institute of Paleontology (Mongolian Academy of Sciences) was involved in the construction of a water pipeline in the eastern Gobi Desert of southeastern Mongolia. Part of this work was conducted near Khanbogd town in Ömnögovi Province. One particular outcrop representing the Bayanshiree Formation ('Urlibe Khudak' locality) yielded the fossil skeleton of a therizinosaurid dinosaur. The excavation of the specimen, led by Idersaikhan Damdinsuren and Ganzorig Bayasgaa with the assistance of other Mongolians, occurred under limited time constraints, restricting the amount of material that could be collected. Furthermore, the specimen was found at the base of a fluvial sandstone bed with pebbles; this high-energy depositional environment, representing a channel lag deposit, also contributed to the incompleteness of the skeleton
    """
    ),
    title="Discovery",
    description="Discovery and naming",
)

In [5]:
root.children = [MarkdownEdge(child=descr), MarkdownEdge(child=discovery)]

# Splitting

In [6]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts.chat import ChatPromptTemplate

from puku_core.graphs.knowledge_trees.types import MarkdownUpdateRequest
from puku_core.documents.markdown import parse, render

In [7]:
from dotenv import load_dotenv

load_dotenv()  # Loads from .env file

True

In [8]:
prompt = ChatPromptTemplate.from_messages(
    messages=[
        (
            "system",
            """You are the best knowledge splitter. You have an article and a few sub-articles. \
                Your task is to split new knowledge between article itself and its edges. For that, you are given of edges description and article itself.
                """,
        ),
        (
            "user",
            """Article: {node}

Edges (or subarticles): each have unique id:
{edges_description}

Amendment to be splitted:
{amendment}
""",
        ),
    ]
)

In [9]:
splitter = ChatModelAmendmentSplitter(
    prompt=prompt, chat_model=ChatDeepSeek(model="deepseek-chat")
)

In [10]:
amendment = """
Prior to its formal description, the fossil material was first reported in two Society of Vertebrate Paleontology conference abstracts in 2015 and 2024.[2][3]

In 2025, Yoshitsugu Kobayashi, Darla Zelenitsky, Anthony Fiorillo, and Tsogtbaatar Chinzorig coauthored a scientific paper in the academic journal iScience. Herein the authors described Duonychus tsogtbaatari as a new genus and species of therizinosaurs, establishing this specimen (MPC-D 100/85) as the holotype. The generic name, Duonychus, combines the Latin word duo, meaning "two" with the Ancient Greek ὄνυξ (ónux), meaning "claw", in reference to the two-fingered hand of the taxon. The specific name, tsogtbaatari, honors Mongolian paleontologist Khishigjav Tsogtbaatar, a former director of the Institute of Paleontology.[1][4]
"""

In [11]:
update_request = MarkdownUpdateRequest(node=root, amendment=parse(amendment))

In [12]:
split = splitter.invoke(update_request)

In [13]:
# print(render(list(split.children.values())[1]))

# Visualizing

In [14]:
from puku.documents.obsidian import dump_knowledge_tree

In [ ]:
# dump_knowledge_tree(node=root, path="/home/curlinkw/workspace/obsidian_vaults/test")